## Initial Setup

In [ ]:
from syft.grid.client.client import connect  # Method used to connect with the domain.
from syft.grid.client.grid_connection import (
    GridHTTPConnection,
)  # Protocol used to talk with the domain

import syft as sy
import torch as th

# Set logging level
import logging

logging.basicConfig(level=logging.INFO)

### Create User Account

In [ ]:
unauthenticated_client = connect(
    url="http://localhost:5000", conn_type=GridHTTPConnection
)

unauthenticated_client.users.create(
    email="scientist@researchorg.edu", password="pwd123"
)

### Log in as Data Scientist

In [ ]:
domain_client = connect(
    url="http://localhost:5000",
    credentials={"email": "scientist@researchorg.edu", "password": "pwd123"},
    conn_type=GridHTTPConnection,
)

### Find a Dataset

In [ ]:
domain_client.datasets.all(pandas=True)

In [ ]:
remote_dataset = domain_client.datasets[0]

In [ ]:
print(remote_dataset.manifest)

In [ ]:
remote_dataset.pandas

### Load Pre-trained Model

In [ ]:
import torch as th
from diabetes_model_training import train_diabetes_model, plot_training_acc

model, loss, acc, epochs_list = train_diabetes_model(th)
plot_training_acc(acc, loss, epochs_list)

### Send the Model to the Worker

In [ ]:
remote_model = model.send(domain_client)

### Perform remote inference

In [ ]:
feature = remote_dataset[0:, 0:8]
labels = remote_dataset[0:, 8]

predicted = remote_model(feature)

### Add DP noise to the evaluation result

In [ ]:
acc = (predicted.reshape(-1).round() == labels).int().tolist()

In [ ]:
predicted.request(reason="I'd like to have access to my accuracy result!")